In [72]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# load the word embeddings into the model from file
filename = 'word2vecSmall.bin.gz'
model = KeyedVectors.load_word2vec_format(filename, binary=True)


In [73]:
# load csv
df = pd.read_csv('../yelp_demo/yelp_reviews.csv')


In [74]:
df['tokenized'] = df['text'].apply(lambda x: x.split())


In [75]:
df['tokenized'] = df['tokenized'].apply(lambda x: [word for word in x if word in model])


In [76]:
df['word_count'] = df['tokenized'].apply(lambda x: len(x))


In [77]:
avg_review_length = int(df['word_count'].mean() - 1)


In [78]:
avg_review_length


73

In [79]:
# split up our dataset into train and test sets using a handy sklearn function
X_train, X_test, y_train, y_test = train_test_split(df['tokenized'].values, df['pos_neg'].values, test_size=0.2)


In [80]:
y_train = np.array([np.array([1.0 if x > 0 else 0.0, 1.0 if x <= 0 else 0.0]) for x in y_train])


In [81]:
y_test = np.array([np.array([1.0 if x > 0 else 0.0, 1.0 if x <= 0 else 0.0]) for x in y_test])


In [82]:
print(len(X_train))

7288


In [83]:
X_train_embeddings = []

for review in X_train:
    if len(review) == 0:
        X_train_embeddings.append(np.zeros((avg_review_length, 300)))
        continue
    embeddings = model[review]
    if len(embeddings) > avg_review_length:
        embeddings = embeddings[:avg_review_length]
    elif len(embeddings) < avg_review_length:
        padding = avg_review_length - len(embeddings)
        padding = np.zeros((padding, 300))
        embeddings = np.concatenate((embeddings, padding))
        X_train_embeddings.append(embeddings)

In [85]:
print(len(X_train_embeddings))

4744


In [61]:
X_test_embeddings = []

for review in X_test:
    if len(review) == 0:
        X_test_embeddings.append(np.zeros((avg_review_length, 300)))
        continue
    embeddings = model[review]
    if len(embeddings) > avg_review_length:
        embeddings = embeddings[:avg_review_length]
    elif len(embeddings) < avg_review_length:
        padding = avg_review_length - len(embeddings)
        padding = np.zeros((padding, 300))
        embeddings = np.concatenate((embeddings, padding))
        X_test_embeddings.append(embeddings)

In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from model import YelpNet

In [67]:
net = YelpNet(300, avg_review_length)

In [68]:
from torch.autograd import Variable

review = Variable(torch.Tensor(X_test_embeddings[0]))
print(review.shape)
prediction = net.forward(review)


torch.Size([73, 300])
tensor([[-0.2891,  0.1992,  0.1602,  ...,  0.1279,  0.1211, -0.2295],
        [-0.5195, -0.0337, -0.0208,  ..., -0.2373, -0.1982, -0.1069],
        [ 0.2021, -0.0811,  0.1836,  ...,  0.0664,  0.0537, -0.1719],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0395], grad_fn=<ReluBackward0>)


In [69]:
print(prediction)


tensor([0.4901, 0.5099], grad_fn=<DivBackward0>)


In [70]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.005)

for i in range(len(X_train_embeddings)):
    try:
        review = Variable(torch.Tensor(X_train_embeddings[i]))
        correct_label = 0 if y_train[i][0] == 1 else 1

        prediction = net.forward(review)

        loss = torch.log(1 + torch.exp(-prediction[correct_label]))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    except:
        print('ouch')
            
torch.save(net, "trained_net.pb")

tensor([[-0.3496,  0.0138, -0.2012,  ..., -0.0566,  0.1855,  0.0747],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        [ 0.0962, -0.0287, -0.1084,  ...,  0.0693,  0.1045, -0.1641],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0514], grad_fn=<ReluBackward0>)
tensor([[-0.0957,  0.2432,  0.2305,  ..., -0.0991,  0.0461,  0.0640],
        [ 0.0070, -0.0732,  0.1719,  ...,  0.0112,  0.1641,  0.1069],
        [ 0.0718,  0.2080, -0.0284,  ..., -0.1680, -0.0209, -0.1426],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0521], grad_fn=<ReluBackward0>)
tensor([[-0.1729,  0.2793,  0.10

tensor([0.0000, 0.0440], grad_fn=<ReluBackward0>)
tensor([[-0.1729,  0.2793,  0.1069,  ...,  0.1230,  0.1299, -0.1826],
        [-0.1650, -0.1670,  0.0479,  ...,  0.0134,  0.1162,  0.2158],
        [ 0.1118,  0.1895,  0.1768,  ..., -0.3145, -0.1104,  0.0269],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0219], grad_fn=<ReluBackward0>)
tensor([[ 0.0791, -0.0050,  0.1118,  ..., -0.0068,  0.0427, -0.1035],
        [-0.1670, -0.0664,  0.0574,  ..., -0.0542,  0.1172,  0.0840],
        [-0.0139, -0.2656,  0.0183,  ..., -0.0425, -0.1455, -0.0708],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0516], grad_fn

tensor([0., 0.], grad_fn=<ReluBackward0>)
tensor([[-0.1729,  0.2793,  0.1069,  ...,  0.1230,  0.1299, -0.1826],
        [-0.1748,  0.0215,  0.2207,  ...,  0.1328,  0.1118,  0.0574],
        [-0.0967, -0.0264,  0.0903,  ..., -0.0197,  0.2139,  0.1367],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0236], grad_fn=<ReluBackward0>)
tensor([[ 0.3711,  0.3789, -0.0957,  ..., -0.4199, -0.2002,  0.1562],
        [-0.0078, -0.0280,  0.0405,  ...,  0.0396, -0.0605,  0.0081],
        [ 0.1406,  0.1436, -0.0461,  ..., -0.1240,  0.1660,  0.0391],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0217], grad_fn=<ReluBa

tensor([[-0.1729,  0.2793,  0.1069,  ...,  0.1230,  0.1299, -0.1826],
        [ 0.0425,  0.3340,  0.0286,  ..., -0.2314,  0.2754,  0.0820],
        [ 0.1235,  0.0320,  0.1504,  ..., -0.2129, -0.0064, -0.0801],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0022], grad_fn=<ReluBackward0>)
tensor([[-0.1729,  0.2793,  0.1069,  ...,  0.1230,  0.1299, -0.1826],
        [-0.2080,  0.1138, -0.0781,  ..., -0.1426,  0.0544,  0.1079],
        [ 0.0942, -0.0228,  0.0522,  ..., -0.0469,  0.1611, -0.1992],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0., 0.], grad_fn=<ReluBackward0>)
tensor([[ 0.2773,  0.1445, -0.0557,  ...

tensor([[-0.0645, -0.0796, -0.1768,  ..., -0.0078,  0.1836, -0.2402],
        [ 0.2793, -0.0148,  0.0679,  ...,  0.0649, -0.1240,  0.2061],
        [-0.1641,  0.0108,  0.0332,  ...,  0.0005,  0.2295, -0.0315],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.0000, 0.0145], grad_fn=<ReluBackward0>)
tensor([[ 0.2949,  0.4961, -0.0055,  ..., -0.2559,  0.2451,  0.1768],
        [-0.1748, -0.1318,  0.1543,  ...,  0.0330,  0.0071,  0.3359],
        [ 0.1094,  0.1406, -0.0317,  ...,  0.0077,  0.1201, -0.1797],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0., 0.], grad_fn=<ReluBackward0>)
tensor([[ 0.0791, -0.0050,  0.1118,  ...

...

tensor([0.9031, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0500,  0.1543,  0.1504,  ...,  0.0874,  0.0159, -0.0229],
        [ 0.2402,  0.2012,  0.1621,  ...,  0.0349,  0.0315,  0.0145],
        [ 0.1094,  0.1406, -0.0317,  ...,  0.0077,  0.1201, -0.1797],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.0982, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0535,  0.1309,  0.2656,  ..., -0.0042,  0.2197, -0.1777],
        [ 0.0664, -0.0242,  0.2324,  ..., -0.0112,  0.0378, -0.2402],
        [ 0.1719,  0.1035,  0.0237,  ...,  0.0004, -0.2295,  0.0425],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.1316, 0.0000], grad_fn

tensor([1.1254, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0170,  0.0522,  0.2041,  ..., -0.0884, -0.0942,  0.0408],
        [-0.0571, -0.0305, -0.0055,  ..., -0.0199,  0.1426, -0.0005],
        [ 0.0801,  0.1050,  0.0498,  ...,  0.0037,  0.0476, -0.0688],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.2105, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0129, -0.0850,  0.0796,  ..., -0.0033,  0.0879,  0.2578],
        [-0.1650,  0.1689, -0.1055,  ...,  0.0325, -0.0140, -0.0356],
        [ 0.0703,  0.0869,  0.0879,  ..., -0.0476,  0.0145, -0.0625],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.7309, 0.0000], grad_fn

tensor([1.0939, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.1035,  0.0131, -0.0825,  ..., -0.1162, -0.1533,  0.0903],
        [-0.0118, -0.0474,  0.0447,  ...,  0.0713, -0.0349,  0.0242],
        [-0.0157, -0.0283,  0.0835,  ...,  0.0069,  0.0610, -0.1484],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.1089, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0791, -0.0050,  0.1118,  ..., -0.0068,  0.0427, -0.1035],
        [-0.1396, -0.0347, -0.0537,  ..., -0.1182, -0.0025, -0.0723],
        [ 0.0938, -0.0933,  0.0544,  ..., -0.0928, -0.0245, -0.0850],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.3637, 0.0000], grad_fn

tensor([[-0.2432, -0.0085,  0.2051,  ...,  0.0337,  0.1289,  0.0708],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        [ 0.1582,  0.1060, -0.1895,  ..., -0.2676,  0.0084, -0.2139],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.3580, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0889,  0.1465,  0.1172,  ..., -0.0109,  0.1484, -0.0564],
        [ 0.1943, -0.0197,  0.0918,  ..., -0.0742,  0.0659, -0.0762],
        [-0.2891,  0.1992,  0.1602,  ...,  0.1279,  0.1211, -0.2295],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.7772, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.2734,  0.1250,  0.08

tensor([[-0.2441,  0.0684,  0.0498,  ...,  0.1982,  0.0693, -0.0698],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        [ 0.1445,  0.0471,  0.1006,  ...,  0.0023,  0.0981, -0.0669],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.1921, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0215, -0.2119, -0.1172,  ..., -0.0903, -0.0544,  0.1797],
        [ 0.1094,  0.1406, -0.0317,  ...,  0.0077,  0.1201, -0.1797],
        [-0.2891,  0.3262,  0.1895,  ...,  0.1377,  0.0938, -0.1592],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.3011, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.1113,  0.2061,  0.22

In [71]:
correct_count = 0
wrong_count = 0

for i in range(len(X_test_embeddings)):
    review = Variable(torch.Tensor(X_test_embeddings[i]))
    correct_label = 0 if y_test[i][0] == 1 else 1
    predictions = net.forward(review)

    # the model's prediction is the digit with the max probability ... how do we find that?
    value, prediction = predictions.max(0)

    if prediction == correct_label:
        correct_count += 1
    else:
        wrong_count += 1

print(correct_count, wrong_count)
print(correct_count / (correct_count + wrong_count))

tensor([[-0.2891,  0.1992,  0.1602,  ...,  0.1279,  0.1211, -0.2295],
        [-0.5195, -0.0337, -0.0208,  ..., -0.2373, -0.1982, -0.1069],
        [ 0.2021, -0.0811,  0.1836,  ...,  0.0664,  0.0537, -0.1719],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.2772, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0791, -0.0050,  0.1118,  ..., -0.0068,  0.0427, -0.1035],
        [-0.0209,  0.0219, -0.0835,  ...,  0.0801,  0.1221, -0.1914],
        [-0.0143,  0.2871, -0.1182,  ..., -0.1216,  0.1367, -0.0540],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.2064, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0535,  0.1309,  0.26

tensor([[ 0.0334,  0.0020,  0.0613,  ..., -0.0148,  0.0216, -0.0330],
        [ 0.0801,  0.1050,  0.0498,  ...,  0.0037,  0.0476, -0.0688],
        [-0.1270,  0.0220,  0.2871,  ..., -0.1016, -0.0903,  0.0884],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.8760, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.1729,  0.2793,  0.1069,  ...,  0.1230,  0.1299, -0.1826],
        [-0.1816,  0.1650, -0.1660,  ...,  0.1592,  0.1279,  0.0022],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.9305, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.2891,  0.1992,  0.16

tensor([[-0.2891,  0.3262,  0.1895,  ...,  0.1377,  0.0938, -0.1592],
        [-0.1006, -0.0247,  0.0923,  ..., -0.1455,  0.2598,  0.0041],
        [-0.0586, -0.0376,  0.0728,  ..., -0.1079, -0.0864, -0.0320],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.5844, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.1396,  0.2314, -0.1309,  ...,  0.0732,  0.2949,  0.3008],
        [-0.0005,  0.1602,  0.0016,  ..., -0.0354,  0.0889, -0.3027],
        [-0.2256, -0.0195,  0.0908,  ...,  0.0282, -0.1777, -0.0060],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.0848, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0535,  0.1309,  0.26

tensor([0.4465, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0796, -0.0767, -0.1631,  ..., -0.1787, -0.0047, -0.0459],
        [-0.2891,  0.1992,  0.1602,  ...,  0.1279,  0.1211, -0.2295],
        [ 0.0070, -0.0732,  0.1719,  ...,  0.0112,  0.1641,  0.1069],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.2863, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.1670,  0.0674,  0.0315,  ..., -0.0025, -0.2295, -0.0344],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        [-0.0476,  0.0815,  0.0457,  ..., -0.0557,  0.1050, -0.1084],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.2175, 0.0000], grad_fn

tensor([1.5811, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0425,  0.0092,  0.1631,  ..., -0.1309, -0.2227,  0.0253],
        [-0.2275,  0.2285,  0.0557,  ...,  0.0815, -0.0115,  0.1729],
        [ 0.0070, -0.0732,  0.1719,  ...,  0.0112,  0.1641,  0.1069],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.4218, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.1172,  0.1426, -0.0986,  ...,  0.1484,  0.0615,  0.1240],
        [ 0.0679, -0.0413,  0.1001,  ...,  0.1045, -0.0854, -0.1543],
        [-0.0762,  0.1270, -0.0156,  ..., -0.0781, -0.0130,  0.1436],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.3889, 0.0000], grad_fn

...

tensor([[-0.1011,  0.0177,  0.0147,  ..., -0.0835,  0.0056, -0.0178],
        [ 0.1104,  0.2480, -0.0077,  ..., -0.0762,  0.1309, -0.2217],
        [-0.0273,  0.0449,  0.0767,  ..., -0.1250,  0.1089, -0.1436],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.2858, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.1504, -0.0566,  0.2012,  ..., -0.1035,  0.1001, -0.0283],
        [-0.0466,  0.0811, -0.0791,  ...,  0.0801, -0.0291,  0.2910],
        [ 0.0493, -0.1001,  0.0067,  ..., -0.0248, -0.1592, -0.0228],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.9423, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0791, -0.0050,  0.11

tensor([[-8.8215e-05,  1.0791e-01, -2.1875e-01,  ...,  1.2256e-01,
          8.1543e-02,  1.4258e-01],
        [ 1.6406e-01, -8.2520e-02, -1.2061e-01,  ...,  1.1523e-01,
         -1.0352e-01, -2.3145e-01],
        [ 1.3086e-01, -8.3496e-02, -9.3262e-02,  ...,  9.4727e-02,
          1.7480e-01,  1.9727e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
tensor([1.4177, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.1582,  0.1060, -0.1895,  ..., -0.2676,  0.0084, -0.2139],
        [ 0.0166,  0.0457, -0.1191,  ...,  0.0165,  0.2637,  0.0845],
        [ 0.0240,  0.3145, -0.0262,  ...,  0.0153,  0.0649,  0.1426],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000

tensor([[-0.1621,  0.0869,  0.2012,  ..., -0.0840,  0.0449, -0.1797],
        [ 0.2041,  0.0132,  0.0757,  ..., -0.2119, -0.1328,  0.1084],
        [-0.1631,  0.3008, -0.1406,  ..., -0.1221,  0.0498,  0.0796],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.3649, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.1729, -0.1045,  0.0698,  ...,  0.0513, -0.1118, -0.4395],
        [-0.0420,  0.1709, -0.1943,  ...,  0.2207, -0.2930, -0.0771],
        [-0.2598,  0.0216,  0.4141,  ...,  0.1416, -0.3770, -0.1348],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.9204, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.1797,  0.1865,  0.00

tensor([[ 0.0791, -0.0050,  0.1118,  ..., -0.0068,  0.0427, -0.1035],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        [-0.0586, -0.0376,  0.0728,  ..., -0.1079, -0.0864, -0.0320],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.3445, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0801,  0.1050,  0.0498,  ...,  0.0037,  0.0476, -0.0688],
        [-0.1816,  0.1650, -0.1660,  ...,  0.1592,  0.1279,  0.0022],
        [ 0.0260, -0.0019,  0.1855,  ..., -0.1216,  0.2217, -0.0220],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([0.9681, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0659, -0.0635, -0.12

tensor([1.0726, 0.0000], grad_fn=<ReluBackward0>)
tensor([[ 0.0703,  0.0869,  0.0879,  ..., -0.0476,  0.0145, -0.0625],
        [ 0.1445,  0.0471,  0.1006,  ...,  0.0023,  0.0981, -0.0669],
        [-0.1089, -0.1846, -0.0520,  ..., -0.1338, -0.1826,  0.1040],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.1739, 0.0000], grad_fn=<ReluBackward0>)
tensor([[-0.0664,  0.1309,  0.0471,  ..., -0.2109,  0.0493,  0.0077],
        [ 0.3145, -0.0026, -0.1670,  ..., -0.2324,  0.1069, -0.0284],
        [ 0.0145,  0.0177, -0.0189,  ..., -0.0977,  0.2305, -0.2178],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([1.1204, 0.0000], grad_fn